In [1]:
import xml.etree.ElementTree as ET
from collections import defaultdict
from os import path, listdir, makedirs, stat
import re
import pickle

def extract_info(dialogue):
    words_tree = ET.parse(path.join(basedata, dialogue + '_words.xml'))
    words_root = words_tree.getroot()
    words = {}
    for word in words_root:
        words[int(word.attrib['id'].strip('word_'))] = word.text
    #moves
    moves_tree = ET.parse(path.join(markables, dialogue + '_move_markables.xml'))
    moves_root = moves_tree.getroot()
    moves_spans = defaultdict(list)
    success_move = {}
    i = 0
    for move in moves_root:
        id_move = move.attrib['span'].replace('word_', '').split('..')
        try:
            success = move.attrib['move_world']
        except:
            success = move.attrib['world']
        if len(id_move) == 2:
            id_move = (int(id_move[0]), int(id_move[1]))
        else:
            id_move = (int(id_move[0]), int(id_move[0]))
        moves_spans[i] = id_move
        success_move[id_move] = success
        i += 1
    utt_tree = ET.parse(path.join(markables, dialogue + '_utterance_markables.xml'))
    utts_root = utt_tree.getroot()
    utts_speaker = defaultdict(list)
    for utt in utts_root:
        span =  utt.attrib['span'].replace('word_', '').split('..')
        if len(span) == 2:
            span = (int(span[0]), int(span[1]))
        else:
            span = (int(span[0]), int(span[0]))
        speaker =  utt.attrib['speaker']
        utts_speaker[speaker].append(span)
    #associate referring expressions to objects
    ref_tree = ET.parse(path.join(markables, dialogue + '_ref_markables.xml'))
    refs_root = ref_tree.getroot()
    refs_objects = defaultdict(list)
    for ref in refs_root:
        ref_type = ref.attrib['referent_type']
        form =  ref.attrib['form']
        span =  ref.attrib['span'].replace('word_', '').split('..')
        if len(span) == 2:
            span = (int(span[0]), int(span[1]))
        else:
            span = (int(span[0]), int(span[0]))
        try:
            item =  ref.attrib['item_id']
            refs_objects[item].append((span, form, ref_type))
        except:
            pass
    moves_utt = defaultdict(dict)
    for s in utts_speaker:
        for utt in utts_speaker[s]:
            for m in moves_spans:
                if utt[0] >= moves_spans[m][0] and utt[0] <= moves_spans[m][1]:
                    if not moves_utt[moves_spans[m]].has_key(s):
                        moves_utt[moves_spans[m]][s] = []
                    moves_utt[moves_spans[m]][s].append(utt)
    moves_ref = defaultdict(dict)
    for obj in refs_objects:
        speaker = ''
        move = ''
        for ref in refs_objects[obj]:
            word_span = ref[0]
            for m in moves_utt:
                for s in moves_utt[m]:
                    for utt in moves_utt[m][s]:
                        if word_span[0] >= utt[0] and word_span[0] <= utt[1]:
                            speaker = s
                            move = m
                            if not moves_ref[move].has_key(speaker):
                                moves_ref[move][speaker] = {}
                            if not moves_ref[move][speaker].has_key(obj):
                                moves_ref[move][speaker][obj] = []
                            moves_ref[move][speaker][obj].append(ref)
                            break
    moves_ref_words = defaultdict(dict)
    for m in moves_ref:
        for s in moves_ref[m]:
            for o in moves_ref[m][s]:
                for ref in moves_ref[m][s][o]:
                    span = ref[0]
                    form = ref[1]
                    ref_type = ref[2]
                    if len(span) == 1:
                        exp = words[span[0]]
                    else:
                        exp = ''
                        i = span[0]
                        while i <= span[1]:
                            if i != span[1]:
                                exp += words[i] + ' '
                            else:
                                exp += words[i]
                            i += 1
                    if not moves_ref_words[m].has_key(s):
                        moves_ref_words[m][s] = {}
                    if not moves_ref_words[m][s].has_key(o):
                        moves_ref_words[m][s][o] = {}
                    if not moves_ref_words[m][s][o].has_key(ref_type):
                        moves_ref_words[m][s][o][ref_type] = []
                    #remove hesitations, and internal tags
                    exp = re.sub(r'<.*>', '', exp)
                    exp = re.sub(r'</.*>', '', exp)
                    exp = re.sub(r'\(.*\)', '', exp)
                    exp = re.sub(r'[\(\)\[\]\.]','', exp)
                    exp = exp.lower()
                    exp_sp = exp.split(' ')
                    if len(exp_sp) == 1 and exp.endswith("'s"):
                        exp = exp.replace("'s", '')
                    moves_ref_words[m][s][o][ref_type].append((exp, span, form))
            for t in moves_ref_words[m][s][o]:
                moves_ref_words[m][s][o][t] = sorted(moves_ref_words[m][s][o][ref_type], key=lambda tup: tup[1][0])
    moves_ref_words = sorted(moves_ref_words.items())
    return moves_ref_words, success_move



In [171]:
#associate word ids to words

data_folder = 'PentoCorpora'
games = listdir(data_folder)
print games
for g in games:
    folder = path.join(data_folder, g)
    basedata = path.join(folder, 'Basedata')
    markables = path.join(folder,'Markables')
    dialogues = []
    output_folder = g.replace('/', '.')
    output_folder = path.join('Dataset', output_folder)
    for doc in listdir(basedata):
        if doc.endswith('_words.xml'):
            dialogue = doc.replace('_words.xml', '')
            dialogues.append(dialogue)
    try:
        stat(output_folder)
    except:
        makedirs(output_folder)
    try:
        stat(output_folder+ '/RefExp')
    except:
        makedirs(output_folder+ '/RefExp')
    try:
        stat(output_folder+ '/Success')
    except:
        makedirs(output_folder+ '/Success')
    for d in dialogues:
        try: 
            r, s = extract_info(d)
            pickle.dump(r, open( path.join(output_folder + '/RefExp', d + "_refexps.p"), "wb" ) )
            pickle.dump(s, open( path.join(output_folder + '/Success', d + "_movesuccess.p"), "wb" ) )
            print d
        except:
            pass

['Disruption_with_Noise_Corpus', 'Push_to_Talk_Corpus', 'Visual_Pento_Corpus']
20061117_run1pento_noise
20061117_run2pento_noise
20061123_pento_nonoise
20070117_run1pento_noise
20070131_run1pento_noise
20070131_run2pento_noise
20070201_run1pento_nonoise
20070201_run3pento_nonoise
20070201_run4pento_nonoise
FTT_2006-02-13pair1
FTT_2006-03-27Pair1
FTT_2006-03-28pair2
FTT_2006-06-22pair1
FTT_2006-06-22pair2
PTT_2006-02-13pair2a
PTT_2006-03-27Pair2a
PTT_2006-03-27pair3a
PTT_2006-03-28pair1
PTT_2006-06-22pair3a
run1605_002b
run1605_003b
run1605_004b
run1605_005b
run1605_006b
run1605_007b


In [2]:
def load_data(dialogue_path):
    with open(dialogue_path) as handle:
        d = pickle.load(handle)
    return d

In [3]:
def move_level(d_refexps):
    new = {}
    for m in d_refexps:
        move, d = m
        for s in d:
            for o in d[s]:
                for r in d[s][o]:
                    for exp in d[s][o][r]:
                        if not new.has_key(move):
                            new[move] = {}
                        if not new[move].has_key((o,r)):
                            new[move][(o,r)] = []
                        text, span, form = exp
                        exp = (span, s, text, form)
                        new[move][(o,r)].append(exp)
        for t in new[move]:
            new[move][t] = sorted(new[move][t], key=lambda tup: tup[0][0])
    new = sorted(new.items())
    return new

In [4]:
def get_previous(refexp, obj_refexps):
    '''
    input: refexp, list of refexps tuples referring to an object
    output: previous refexp by different speaker with that object
    '''
    i = obj_refexps.index(refexp) - 1
    speaker = refexp[1]
    previous = 'no_previous'
    while i >= 0:
        if obj_refexps[i][1] != speaker:
            previous = obj_refexps[i]
            break
        i =- 1
    return previous

In [39]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('german')
ignore_words = stopwords.words('german')
ignore_words.extend(['dis', "'n"])
from string import punctuation
punct = list(punctuation)

def lex_material(exp):
    exp = exp[2]
    words = word_tokenize(exp) 
    content_words = set()
    for w in words:
        w = stemmer.stem(w)
        if not w in punct:
            stopword = False
            for sw in ignore_words:
                if sw == w:
                    stopword = True
            if stopword == False:
                content_words.add(w)
    return content_words

def lex_sim(exp1, exp2):
    words1 = lex_material(exp1)
    words2 = lex_material(exp2)
    try:
        diff = 1 - float(2*len(words1.intersection(words2)))/float(len(words1)+ len(words2))
        print diff
    except:
        diff = 'undefined'
    return dice_sim

In [40]:
def gamerun_level(d_refexps):
    new = {}
    for m in d_refexps:
        move, d = m
        for s in d:
            for o in d[s]:
                for r in d[s][o]:
                    for exp in d[s][o][r]:
                        if not new.has_key((o,r)):
                            new[(o,r)] = []
                        text, span, form = exp
                        exp = (span, s, text, form)
                        new[(o,r)].append(exp)
    for t in new:
        new[t] = sorted(new[t], key=lambda tup: tup[0][0])
    return new

In [41]:
import numpy as np
from collections import defaultdict
dialogue = 'Dataset/Push_to_Talk_Corpus/RefExp/FTT_2006-02-13pair1_refexps.p'
d = load_data(dialogue)
refexps = gamerun_level(d)
diffs = []
for o in refexps:
    refobj = refexps[o]
    i = 1
    while i< len(refobj):
        if get_previous(refobj[i], refobj) != 'no_previous':
            diff = lex_sim(refobj[i], get_previous(refobj[i], refobj))
            #print sim, refobj[i], get_previous(refobj[i], refobj), o
            if diff != 'undefined':
                diffs.append(sim)
        i += 1
print np.mean(diffs)
#print refobj
#refexps = gamerun_level(d)

'''
def player_level(d_refexps): use the other function
def lex_sim(ref_exps):
'''

1.0


NameError: global name 'dice_sim' is not defined